1. Init scirpt parameters

In [ ]:
# Set SingleNode Training or MultiNode Training
MULTI_NODE = True

In [ ]:
# Input your project id 
PROJECT_ID = <PROJECT_ID>  # @param {type:"string"}

In [ ]:
# Input your GCP resource region
REGION = <REGION>  # @param {type: "string"}

In [ ]:
# Input your cloud bucket URI, example: gcs://<bucketname>
BUCKET_URI = <GCS_BUCKET_URI>  # @param {type:"string"}

In [ ]:
# Input your custom training image uri
IMAGE_URI= <IMAGE_URI> # @param {type:"string"}

In [ ]:
# Input Vertex AI tensorboard display name
TENSORBOARD_DISPLAY_NAME = "llama2_tensorboard" # @param {type:"string"}

In [ ]:
# Input service account, please create the service acccount before the training
SVC_ACCOUNT=<SVC_ACCOUNT> # @param {type:"string"}

2. Create Service Account

3. Create Cloud Bucket

4. Create Vertex AI tensorboard instance

In [ ]:
# Create tensorboard instance
from google.cloud import aiplatform
def create_tensorboard_sample(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    tensorboard = aiplatform.Tensorboard.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(tensorboard.resource_name)
    return tensorboard.resource_name

In [ ]:
# input Vertex AI tensorboard instance id, example projects/<project_number>/locations/<region>/tensorboards/<tensorboard_instance_id>
# TENSORBOARD= <TENSORBOARD_INSTANCE_ID> # @param {type:"string"}
TENSORBOARD = create_tensorboard_sample(project=PROJECT_ID, display_name=TENSORBOARD_DISPLAY_NAME, location=REGION)

5. Build custom training image

6. Create Vertex AI custome training JOB

In [ ]:
# Input cluster spec parameters
HEAD_NODE_ACCELERATOR_COUNT = 1
HEAD_NODE_COUNT = 1

# If multinode training, set worker node spec parameters
WORKER_NODE_ACCELERATOR_COUNT = 1
WORKER_NODE_COUNT = 1

In [ ]:
# Setup python package dependacy
from datetime import datetime
from google.cloud import aiplatform

# init aiplaform sdk
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=REGION)

In [ ]:
# Execute this cell to launch an SingleNode training
if MULTI_NODE == False:
    worker_pool_specs = [
        # `WorkerPoolSpec` for worker pool 0, primary replica, required  
        {
            "machine_spec": {
                "machine_type": "a2-highgpu-1g", # "a2-highgpu-1g","g2-standard-12","n1-standard-4"
                "accelerator_type": "NVIDIA_TESLA_A100", #"NVIDIA_L4",
                "accelerator_count": HEAD_NODE_ACCELERATOR_COUNT,       
            },
            "replica_count": HEAD_NODE_COUNT,
            "container_spec": {
                "image_uri": IMAGE_URI,
                "command": [],
                "args": [],
                "env": [],                
            },
            "disk_spec": {
                "boot_disk_size_gb": 1000,            
            }
        },
    ]

    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    JOB_NAME  = "llama2finetunecustomjob " + TIMESTAMP

    my_job = aiplatform.CustomJob(
        display_name=JOB_NAME,    
        worker_pool_specs=worker_pool_specs,
        base_output_dir=BUCKET_URI,
    )

In [ ]:
# Execute this cell to launch an MultiNode training
if MULTI_NODE == True:
    worker_pool_specs = [
        # `WorkerPoolSpec` for worker pool 0, primary replica, required  
        {
            "machine_spec": {
                "machine_type": "g2-standard-12", # "a2-highgpu-1g","g2-standard-12","n1-standard-4"
                "accelerator_type": "NVIDIA_TESLA_A100", #"NVIDIA_L4",
                "accelerator_count": HEAD_NODE_ACCELERATOR_COUNT,       
            },
            "replica_count": HEAD_NODE_COUNT,
            "container_spec": {
                "image_uri": IMAGE_URI,
                "command": [],
                "args": [],
                "env": [],                
            },
        },
        {
        "machine_spec": {
                "machine_type": "g2-standard-12", # "a2-highgpu-1g","g2-standard-12","n1-standard-4"
                "accelerator_type": "NVIDIA_TESLA_A100", #"NVIDIA_L4",
                "accelerator_count": WORKER_NODE_ACCELERATOR_COUNT,          
        },
        "replica_count": WORKER_NODE_COUNT,        
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": [],
                "args": [],
                "env": [],
        },        
        "disk_spec": {
                "boot_disk_size_gb": 1000,            
        }        
        },
    ]

    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    JOB_NAME  = "llama2finetunecustomjob " + TIMESTAMP

    my_job = aiplatform.CustomJob(
        display_name=JOB_NAME,    
        worker_pool_specs=worker_pool_specs,
        base_output_dir=BUCKET_URI,
    )

In [ ]:
# Submit Vertex AI custom trainig JOB
my_job.submit(    
    enable_web_access=True, # For debugging
    service_account=SVC_ACCOUNT,
    tensorboard=TENSORBOARD,
)